In [93]:
import os
import time
import paramiko
from midi_player import MIDIPlayer
from midi_player.stylers import dark

In [94]:
# Configuration
remote_host = "129.173.66.44"
remote_folder = "/home/finlay/disklavier/data/outputs"
local_file_path = os.path.join(
    "../data/datasets/20240606/play", os.listdir("../data/datasets/20240606/play")[0]
)
local_folder = os.path.dirname(local_file_path)

assert os.path.exists(local_file_path)
MIDIPlayer(local_file_path, 400, styler=dark)

In [95]:
# Create SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(remote_host, 22, "finlay")

# Create SFTP client
sftp = ssh.open_sftp()

In [96]:
# Check if the file already exists on the remote server
try:
    sftp.stat(remote_file_path)  # Try to get the file's status
    # If the file exists, delete it
    sftp.remove(remote_file_path)
    sftp.remove(remote_file_path[:-4] + ".pt")
    print(f"Existing file '{remote_file_path}' deleted.")
except FileNotFoundError:
    # File does not exist, no action needed
    print(f"No existing file found at '{remote_file_path}'. Proceeding with upload.")
# Upload the file
remote_file_path = os.path.join(remote_folder, os.path.basename(local_file_path))
sftp.put(local_file_path, remote_file_path)

Existing file '/home/finlay/disklavier/data/outputs/20240126-050-04_0335-0345.mid' deleted.


<SFTPAttributes: [ size=527 uid=1011 gid=1011 mode=0o100664 atime=1728957454 mtime=1728957454 ]>

In [97]:
fp_tensor = remote_file_path[:-4] + ".pt"
while 1:
    try:
        print(f"looking for '{fp_tensor}'")
        sftp.stat(fp_tensor)
        break
    except FileNotFoundError:
        time.sleep(1)

looking for '/home/finlay/disklavier/data/outputs/20240126-050-04_0335-0345.pt'
looking for '/home/finlay/disklavier/data/outputs/20240126-050-04_0335-0345.pt'


In [98]:
# Download the new file
sftp.get(fp_tensor, os.path.join(local_folder, os.path.basename(new_file_path)))

In [99]:
# Clean up
sftp.close()
ssh.close()